## The MNIST Dataset

The MNIST dataset comprises 60,000 training images and 10,000 testing images of handwritten digits. These images are in grayscale and have a dimension of 28x28 pixels. In this section, our goal is to create a Feed Forward Neural Network that can effectively categorize these images into the digits 0 to 9.
![digit demo](digit_demo.png)

In this tutorial, our focus is on creating a feedforward neural network designed to categorize handwritten digits. These images of handwritten digits are part of a well-known dataset known as MNIST, which is highly renowned in the field.

Our approach involves flattening the images, essentially transforming them into a one-dimensional format. The number of input neurons in the neural network corresponds to the total number of pixels (28*28 = 784) in each image.

For the output layer of our neural network, we have ten neurons. This number aligns with the ten classes we aim to classify because we're dealing with digits ranging from 0 to 9. In essence, our network's purpose is to classify these ten different digits.


In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

We imported ***torchvision.datasets*** for:

### Accessing Standard Datasets:
***torchvision.datasets*** provides a collection of widely-used standard datasets for computer vision tasks. One of the most famous datasets is the MNIST dataset, which contains images of handwritten digits. Other datasets, like CIFAR-10, CIFAR-100, and ImageNet, are also available. This module makes it easy to access and load these datasets into your project without having to manually download and preprocess the data.

### Data Loading and Transformation:
The datasets module includes functions and classes to load and preprocess the data. It allows you to apply various transformations to the images, such as resizing, cropping, and data augmentation. This is crucial for preparing the data in a format that's suitable for training deep learning models.

In [ ]:
input_size = 784      # Number of input neurons (image pixels)
hidden_size = 400     # Number of hidden neurons
out_size = 10         # Number of classes (0-9)
epochs = 10           # How many times we pass our entire dataset into our network
batch_size = 100      # Input size of the data during one iteration
learning_rate = 0.001 # How fast we are learning

In [ ]:
train_dataset = datasets.MNIST(root='./',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = datasets.MNIST(root='./',
                             train=False,
                             transform=transforms.ToTensor())

In [ ]:
# make data iterable by loading it to a loader, shuffle the training data to make it independent of the order.
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
# when we want to divide our input size into batches, then we need to use train loader

![digit neural network](digit_nn.png)

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)   # first layer
        self.fc2 = nn.Linear(hidden_size, hidden_size)      # second layer
        self.fc3 = nn.Linear(hidden_size, out_size)         # third layer
        self.relu = nn.ReLU()                               # activation function

        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [ ]:
# Create an object of the class, which represents our network
net = Net(input_size, hidden_size, out_size) 
CUDA = torch.cuda.is_available()
if CUDA:
    #if gpu is available, then switch the network to gpu to work faster
    net = net.cuda()
    
# the loss function. The Corss Entropy loss comes along with softmax. Therefore, no need to specify softmax as well
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
# let's see what are inside the net.parameters. 
net.parameters
# list(net.parameters())

In [ ]:
# visualize the train loader
for i, (images, labels) in enumerate(train_loader):
    print(images.size()) 
    images = images.view(-1, 784) #reshaping the tensor
    print(images.size())

**torch.Size([100, 1, 28, 28])** ==>  100 corresponds to the batch size that we define earlier. So in each iterations, there's 100 images. 1 corresponnds to the number of channels in the image, since we have a grayscaled imaged(not coloured image), if the number of channels in the image is 3 then we have an RGB or colored image. 28 is the height or the number of rows, and the other 28 is the number of columns or the width of the image.

In [ ]:
# Train the network
correct_train = 0                                # How many correct classifications our network classifies out of total_train samples
total_train = 0
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Flatten the image from size (batch,1,28,28) --> (100,1,28,28) where 1 represents the number of channels (grayscale-->1),
        # to size (100, 784) and wrap it in a variable
        images = Variable(images.view(-1,28*28))
        labels = Variable(labels)
        
        #switching between cpu and gpu
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()
        
        # clear the param_grad in param <-- param - lr*param_grad, so it won't accumulated
        optimizer.zero_grad() 
        
        outputs = net(images)                      # forward pass, 
        _, predicted = torch.max(outputs.data, 1)  # first argument returns the maximum score and second argument represents the index location of each maximum value found
        
        total_train += labels.size(0)              # labels.size(0) = 100 (batch_size)
        
        if CUDA:
            # if cuda is available, then all calculations are happening in gpu. so we need to
            # transfer these tensors to a cpu because sum() is a python function, there is
            # no implementation of sum() on a gpu
            correct_train += (predicted.cpu() == labels.cpu()).sum()
        else:
            # if we don't have cuda
            correct_train += (predicted == labels).sum()
            
        loss = criterion(outputs, labels)           # difference between the actual and predicted(loss function)
        loss.backward()                             # Backpropagation
        optimizer.step()                            # update the weights
        
        if (i+1) % 100 == 0:                        # printing the results
            print('Epoch [{}/{}], Iteration [{}/{}], Training Loss: {}, Training Accuracy: {}%'.format
                  (epoch+1, epochs, i+1, len(train_dataset)//batch_size, loss.item(), (100*correct_train/total_train)))
print("\n\nDONE TRAINING !!")        

correct_train --> ow many correct classifications our network classifies out of total_train samples. Through every iteration in the loop, the total_train will be incremented by the batch size(100)

There are 2 loops:
 - The outer loop iterates over epochs.
 - The inner loop iterates over batches of data. Within each epoch, the training dataset is divided into smaller batches, and the inner loop processes these batches one by one. **enumerate(train_loader)** is used to obtain batches of data from the **train_loader**. In each iteration of the inner loop, a batch of images and their corresponding labels are loaded and processed. The neural network is updated with each batch of data using backpropagation and gradient descent.

*labels* are 0 to 9

The purpose of using **Variable** was to enable automatic differentiation. By wrapping tensors with **Variable**, you signaled to PyTorch that you wanted to track operations on those tensors and compute gradients. This is important because gradients are necessary for updating model parameters during training with gradient-based optimization algorithms like stochastic gradient descent (SGD).

However, since PyTorch 0.4 and later versions, automatic differentiation is enabled by default on tensors, which means that you can perform operations on tensors directly, and PyTorch will automatically track those operations and compute gradients. In these newer versions, using Variable for most common use cases is no longer required, and you can work with tensors directly.

**outputs**, contains the network's predictions or output for the given input batch of images. After obtaining the network's output (**outputs**), we typically want to determine which class or category the network has predicted for each input and **torch.max(outputs.data, 1)** does that task.
**torch.max(outputs.data, 1)** finds the maximum value along dimension 1 (columns) of the outputs tensor. Maximum value basically indicates which label is classified or predicted by the network.

In [ ]:
# Test the network (We need No loss and weight calculation, no weight update, 
# just forward propagation is needed)
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    
    if CUDA:
        images = images.cuda()
    # For each input (or, sample or, row) in the batch, the output will contain 10 elements
    outputs = net(images)
    # We could also write: predicted = outputs.data.max(1)[1]
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)         # Increment the total count(100)
    # We can also use: correct := predicted.eq(labels).sum()
    if CUDA:
        correct += (predicted.cpu() == labels.cpu()).sum()
    else:
        correct += (predicted == labels).sum()
        
print("Final Test Accuracy is : %d %%" % (100*correct/total))
    

In our project, our feedforward neural network has achieved an impressive 97% accuracy on the test dataset, indicating that it doesn't suffer from overfitting and can generalize well to new, unseen data. We plan to explore convolutional neural networks in the next section to investigate the potential for further improvement beyond this already high accuracy. While the current accuracy is commendable, there is still room for enhancement, and we intend to employ techniques like batch normalization and dropout in the upcoming section to potentially push the accuracy even higher. We anticipate that the application of these techniques within the context of convolutional networks will lead to an increase in our final test accuracy, further enhancing the performance of our model.